In [ ]:
ak = ""
sk = ""

In [ ]:
import json
from huaweicloudsdkcore.auth.credentials import GlobalCredentials
from huaweicloudsdkiam.v5.region.iam_region import IamRegion
from huaweicloudsdkiam.v5 import IamClient, ListUsersV5Request, ListAttachedUserPoliciesV5Request

def get_all_iam_users_and_policies():

    if not ak or not sk:
        raise ValueError("请设置环境变量 HUAWEICLOUD_SDK_AK 和 HUAWEICLOUD_SDK_SK")

    credentials = GlobalCredentials(ak, sk)  # 无需 project_id

    # 2. 构建 IAM 客户端（指定任意 Region，SDK 自动处理全局 endpoint）
    client = (IamClient.new_builder() 
        .with_credentials(credentials)
        .with_region(IamRegion.CN_NORTH_4) 
        .build())

    # 3. 获取所有用户（处理分页）
    all_users = []
    marker = None
    while True:
        request = ListUsersV5Request()
        request.limit = 100  # 每页最大 100
        if marker:
            request.marker = marker

        response = client.list_users_v5(request)
        if response.status_code != 200:
            raise Exception(f"获取用户列表失败: {response}")

        users_data = response.to_json_object()
        users = users_data.get("users", [])
        all_users.extend(users)

        # 分页：检查 next_marker
        page_info = users_data.get("page_info", {})
        marker = page_info.get("next_marker")
        if not marker:
            break

    print(f"总用户数: {len(all_users)}")

    # 4. 对于每个用户，获取附加的权限策略（处理分页）
    users_with_policies = []
    for user in all_users:
        user_id = user["user_id"]
        user_name = user["user_name"]
        user_info = {
            "user_id": user_id,
            "user_name": user_name,
            "description": user.get("description", ""),
            "enabled": user.get("enabled", False),
            "created_at": user.get("created_at", ""),
            "policies": []  # 权限列表
        }

        policy_marker = None
        while True:
            policy_request = ListAttachedUserPoliciesV5Request(user_id=user_id)
            policy_request.limit = 100
            if policy_marker:
                policy_request.marker = policy_marker

            policy_response = client.list_attached_user_policies_v5(policy_request)
            if policy_response.status_code != 200:
                print(f"警告: 获取用户 {user_name} 的权限失败: {policy_response}")
                break

            policies_data = policy_response.to_json_object()
            policies = policies_data.get("attached_policies", [])
            user_info["policies"].extend(policies)

            # 分页
            policy_page_info = policies_data.get("page_info", {})
            policy_marker = policy_page_info.get("next_marker")
            if not policy_marker:
                break

        users_with_policies.append(user_info)

    # 5. 输出结果（JSON 格式）
    result = {"users": users_with_policies}
    print(json.dumps(result, indent=2, ensure_ascii=False))
    return result

if __name__ == "__main__":
    get_all_iam_users_and_policies()